In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#from subprocess import check_output
#print(check_output(["ls", "../input/batchtestbis/fBATCH"]).decode("utf8"))


# Passenger Screening Challenge Kaggle

## Summary

## Introduction
In this competition, TSA is stepping outside their established procurement process and is challenging the broader data science community to help improve the accuracy of their threat prediction algorithms. Using a dataset of images collected on the latest generation of scanners, participants are challenged to identify the presence of simulated threats under a variety of object types, clothing types, and body types. Even a modest decrease in false alarms will help TSA significantly improve the passenger experience while maintaining high levels of security.

    _.aps = projected image angle sequence file (10.3MB per file)
    _.a3d = combined image 3D file (330MB per file)
    _.a3daps = combined image angle sequence file (41.2MB per file)
    _.ahi = calibrated object raw data file (2.26GB per file)
    
The data type of this file is 16bit unsigned integer. Data scaling is achieved by multiplying each pixel value by the 'data_scale_factor' field in the header. 

## Libraries

In [ ]:
import pandas as pd #used for the data frame manipulation
import numpy as np #used for the array manipulation
import os #used for the interface operating system

import sklearn as skl #used for machine learning algorithm
from sklearn.cluster import MeanShift, estimate_bandwidth

import matplotlib #used for the visualisation
import matplotlib.pyplot as plt #used for the style of Matlab function
import matplotlib.animation as anim # used for life animation of Matplotlib

%matplotlib inline

In [ ]:
os.sys.path

In [ ]:
import platform
platform.architecture()

In [ ]:
#import cv2

## Macro Variables

In [ ]:
pathData = os.path.join("../input")
dataTrain = os.path.join(pathData,"passenger-screening-algorithm-challenge/stage1_labels.csv")
dataSub = os.path.join(pathData,"passenger-screening-algorithm-challenge/stage1_sample_submission.csv")
dataAps= os.path.join(pathData,"batchtest")
dataApsBis= os.path.join(pathData,"batchtestbis")

## Macro functions

In [ ]:
# To read the header of the files
def read_header(infile):
    """Read image header (first 512 bytes)
    """
    h = dict()
    fid = open(infile, 'rb')#+b
    h['filename'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 20))
    h['parent_filename'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 20))
    h['comments1'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 80))
    h['comments2'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 80))
    h['energy_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['config_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['file_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['trans_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['scan_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['data_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['date_modified'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 16))
    h['frequency'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['mat_velocity'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['num_pts'] = np.fromfile(fid, dtype = np.int32, count = 1)
    h['num_polarization_channels'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['spare00'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['adc_min_voltage'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['adc_max_voltage'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['band_width'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['spare01'] = np.fromfile(fid, dtype = np.int16, count = 5)
    h['polarization_type'] = np.fromfile(fid, dtype = np.int16, count = 4)
    h['record_header_size'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['word_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['word_precision'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['min_data_value'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['max_data_value'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['avg_data_value'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['data_scale_factor'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['data_units'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['surf_removal'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['edge_weighting'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['x_units'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['y_units'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['z_units'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['t_units'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['spare02'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['x_return_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_return_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_return_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['scan_orientation'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['scan_direction'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['data_storage_order'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['scanner_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['x_inc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_inc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_inc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['t_inc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['num_x_pts'] = np.fromfile(fid, dtype = np.int32, count = 1)
    h['num_y_pts'] = np.fromfile(fid, dtype = np.int32, count = 1)
    h['num_z_pts'] = np.fromfile(fid, dtype = np.int32, count = 1)
    h['num_t_pts'] = np.fromfile(fid, dtype = np.int32, count = 1)
    h['x_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['x_acc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_acc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_acc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['x_motor_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_motor_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_motor_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['x_encoder_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_encoder_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_encoder_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['date_processed'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 8))
    h['time_processed'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 8))
    h['depth_recon'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['x_max_travel'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_max_travel'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['elevation_offset_angle'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['roll_offset_angle'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_max_travel'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['azimuth_offset_angle'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['adc_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['spare06'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['scanner_radius'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['x_offset'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_offset'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_offset'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['t_delay'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['range_gate_start'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['range_gate_end'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['ahis_software_version'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['spare_end'] = np.fromfile(fid, dtype = np.float32, count = 10)
    return h

In [ ]:
# To read the data of the files
def read_data(infile):
    """Read any of the 4 types of image files, returns a numpy array of the image contents
    """
    extension = os.path.splitext(infile)[1]
    h = read_header(infile)
    nx = int(h['num_x_pts'])
    ny = int(h['num_y_pts'])
    nt = int(h['num_t_pts'])
    fid = open(infile, 'rb')
    fid.seek(512) #skip header
    if extension == '.aps' or extension == '.a3daps':
        if(h['word_type']==7): #float32
            data = np.fromfile(fid, dtype = np.float32, count = nx * ny * nt)
        elif(h['word_type']==4): #uint16
            data = np.fromfile(fid, dtype = np.uint16, count = nx * ny * nt)
        data = data * h['data_scale_factor'] #scaling factor
        data = data.reshape(nx, ny, nt, order='F').copy() #make N-d image
    elif extension == '.a3d':
        if(h['word_type']==7): #float32
            data = np.fromfile(fid, dtype = np.float32, count = nx * ny * nt)
        elif(h['word_type']==4): #uint16
            data = np.fromfile(fid, dtype = np.uint16, count = nx * ny * nt)
        data = data * h['data_scale_factor'] #scaling factor
        data = data.reshape(nx, nt, ny, order='F').copy() #make N-d image
    elif extension == '.ahi':
        data = np.fromfile(fid, dtype = np.float32, count = 2* nx * ny * nt)
        data = data.reshape(2, ny, nx, nt, order='F').copy()
        real = data[0,:,:,:].copy()
        imag = data[1,:,:,:].copy()
    fid.close()
    if extension != '.ahi':
        return data
    else:
        return real, imag

In [ ]:
# To build the animation of Matplotlib
matplotlib.rc('animation', html='html5')

def plot_image(path):
    data = read_data(path)
    fig = matplotlib.pyplot.figure(figsize = (16,16))
    ax = fig.add_subplot(111)
    def animate(i):
        im = ax.imshow(np.flipud(data[:,:,i].transpose()), cmap = 'viridis')
        return [im]
    return matplotlib.animation.FuncAnimation(fig, animate, frames=range(0,data.shape[2]), interval=200, blit=True)

## Import Data

In [ ]:
stgdf= pd.read_csv(dataTrain)
stgdf.head()


## Transform Data

In [ ]:
stgdf['Ind'], stgdf['Zone'] = stgdf['Id'].str.split('_Zone', 1).str
stgdf['Zone']=stgdf['Zone'].apply(pd.to_numeric)
stgdf.head()

## Explore Data

81% of the individuals are reported at least with one dangerous object. It represents for 928 individuals out of 1147.

In [ ]:
df=stgdf.groupby('Ind')[['Probability']].max()
df['Probability'].describe()
df['Probability'].value_counts()

Let look now which zone is holding more dangerous object

In [ ]:
df=stgdf.groupby('Zone',as_index=False)[['Probability']].sum()
df.sort_values(['Probability'],ascending=False)
#plt.bar(df.Zone, df.Probability, align='center', alpha=0.5)
#plt.ylabel('Usage')
#plt.title('Programming language usage')

## Open aps files



### List aps file names

In [ ]:
names_file=stgdf['Ind'].unique()
names_file=names_file+'.aps'
folder_names_file=[str(x[0])+'BATCH/' for x in names_file]
folder_names_file=[m+n for m,n in zip(folder_names_file,names_file)]
len(names_file)
import random
c = list(zip(names_file, folder_names_file))
random.shuffle(c)
names_file, folder_names_file = zip(*c)
nind=50
ratio=0.9

In [ ]:
for zip_file in c[:nind]:
    #names_file, folder_names_file=zip(*zip_file)
    print(zip_file[0])
    print(zip_file[1])

In [ ]:
names_file=names_file[:nind]
folder_names_file=folder_names_file[:nind]

### Limited List aps file of nind random persons

In [ ]:
data_file = {zip_file[0] : read_data(os.path.join(dataApsBis,zip_file[1])) if zip_file[0][0]=='f' else read_data(os.path.join(dataAps,zip_file[1])) for zip_file in c[:nind]}

### Animation of the .aps file

Regarding the nature of the image of type _.aps

    The 'Projected Image' algorithm computes 3D images for 90-degree segments of data that are equally spaced around the region scanned. A maximum value projection of the result of each of these computations is written sequentially into a single file. The result of this is an image file that, when played back plane-by-plane, appears like the object is spinning on the screen.

    Data file order: AYX (angle, vertical axis, horizontal axis)
    Axis Name, Stride, Number of samples, Axis Length
    XAxis, 1, Nx=512, Lx=1.0 meters
    YAxis, 512, Ny=660, Ly=2.0955 meters
    Angular, 337920, Na=16, La=360-degrees

    The data type of this file is 16bit unsigned integer. Data scaling is achieved by multiplying each pixel value by the 'data_scale_factor' field in the header. In summary, the 'Projected Image Angle Sequence' file is a series of 2D mmWave snapshots equally spaced in angle around the object.

In [ ]:
k=0
data_file1=data_file[c[k][0]]
if c[k][0][0]=='f' :
    read_header(os.path.join(dataApsBis,c[k][1]))
else :
    read_header(os.path.join(dataAps,c[k][1]))
    

In [ ]:
a=np.asarray(stgdf[stgdf.Ind==names_file[k].replace('.aps', '')].sort_values(by=['Zone'])['Probability'])
b=stgdf[stgdf.Ind==names_file[k].replace('.aps', '')].sort_values(by=['Zone'])
a

In [ ]:
#Based on the tflearn example located here:
#https://github.com/tflearn/tflearn/blob/master/examples/images/convnet_cifar10.py

from __future__ import division, print_function, absolute_import

# Import tflearn and some helpers
import tensorflow as tf
import tflearn
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
from tflearn.data_utils import shuffle
import pickle

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten, Dropout
from keras.models import Model
from keras.layers import concatenate

In [ ]:
nindl=range(nind)
nindb=int(ratio*nind)
trY = np.asarray([stgdf[stgdf.Ind==names_file[k].replace('.aps', '')].sort_values(by=['Zone'])['Probability']
                  for k in nindl[:nindb]])
teY = np.asarray([stgdf[stgdf.Ind==names_file[k].replace('.aps', '')].sort_values(by=['Zone'])['Probability']
                  for k in nindl[nindb:]])

In [ ]:
trX = dict()
tData=dict((k, data_file[k]) for k in names_file[:nindb])
for i in range(16):
    Xi = [value[:,:,i].transpose() for value in tData.values()]
    Xi=np.asarray(Xi)/np.amax((np.abs(np.asarray(Xi))))
    Xi = Xi.reshape(Xi.shape[0], 660, 512,1)
    trX[i]=Xi

teX = dict()
tData=dict((k, data_file[k]) for k in names_file[nindb:])
for i in range(16):
    Xi = [value[:,:,i].transpose() for value in tData.values()]
    Xi=np.asarray(Xi)/np.amax((np.abs(np.asarray(Xi))))
    Xi = Xi.reshape(Xi.shape[0], 660, 512,1)
    teX[i]=Xi

In [ ]:
del data_file

In [ ]:
class_X = dict()
for i in range(17):
    Xi=dict()
    Yi=trY[:,i]
    zero_weight=sum(Yi)
    uno_weight=Yi.shape[0]-zero_weight
    zero_weight /=Yi.shape[0]
    uno_weight/=Yi.shape[0]
    Xi[0]=max(zero_weight/uno_weight,0.05)
    Xi[1]=min(uno_weight/zero_weight,20)
    class_X[int(i+1)]=Xi

In [ ]:
class_X

In [ ]:
class ConvNet( object ):

    def parameters(self): 
        params_w = {'wLyr1': tf.Variable(tf.random_normal([ 5, 5, 1,  self.lyr1FilterNo_                        ])),
                    'wLyr2': tf.Variable(tf.random_normal([ 3, 3,     (self.imageRot_)*self.lyr1FilterNo_ , self.lyr2FilterNo_   ])),
                    'wLyr3': tf.Variable(tf.random_normal([ 3, 3,     self.lyr2FilterNo_   , self.lyr2FilterNo_   ])),
                    'wFCh1':  tf.Variable(tf.random_normal([ self.LyrSize_ , self.fcHidLyrSize1_   ])),
                    'wFCh2':  tf.Variable(tf.random_normal([ self.fcHidLyrSize1_           , self.fcHidLyrSize2_   ])),   
                    'wFCh3':  tf.Variable(tf.random_normal([ self.fcHidLyrSize2_           , self.fcHidLyrSize3_   ])),
                    'wFCh4':  tf.Variable(tf.random_normal([ self.fcHidLyrSize3_           , self.fcHidLyrSize4_   ])),
                    'wFCh5':  tf.Variable(tf.random_normal([ self.fcHidLyrSize4_           , self.fcHidLyrSize5_   ])),
                    'wFCh6':  tf.Variable(tf.random_normal([ self.fcHidLyrSize5_           , self.fcHidLyrSize6_   ])),
                    'wOut':  tf.Variable(tf.random_normal([  self.fcHidLyrSize6_            , self.outLyrSize_     ]))}
                
        params_b = {'bLyr1': tf.Variable(tf.random_normal([           self.lyr1FilterNo_                        ])),
                    'bLyr2': tf.Variable(tf.random_normal([           self.lyr2FilterNo_                        ])),
                    'bLyr3': tf.Variable(tf.random_normal([           self.lyr2FilterNo_                        ])),
                    'bFCh1':  tf.Variable(tf.random_normal([           self.fcHidLyrSize1_                        ])),
                    'bFCh2':  tf.Variable(tf.random_normal([           self.fcHidLyrSize2_                        ])),
                    'bFCh3':  tf.Variable(tf.random_normal([           self.fcHidLyrSize3_                        ])),
                    'bFCh4':  tf.Variable(tf.random_normal([           self.fcHidLyrSize4_                        ])),
                    'bFCh5':  tf.Variable(tf.random_normal([           self.fcHidLyrSize5_                        ])),
                    'bFCh6':  tf.Variable(tf.random_normal([           self.fcHidLyrSize6_                        ])),
                    'bOut':  tf.Variable(tf.random_normal([           self.outLyrSize_                          ]))}
        return params_w,params_b

    #======================================================================================================================================================================================================================    
    
    def scoreB(self):
        
        def scoreA(xi,W, b, strides=1,k=2):
            x = tf.reshape(xi, shape = [-1,660,512,1])
            x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='VALID')
            x = tf.nn.bias_add(x, b)
            x = tf.nn.relu(x)   
            x= tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')
            return x
        #------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------      

        def conv2d(x, W, b, strides=1):
            x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='VALID')
            x = tf.nn.bias_add(x, b)
            return x

        #------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  

        def maxpool2d(x, k=2):
            return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')
        #------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        output1 = scoreA (self.x1_, self.params_w_['wLyr1'], self.params_b_['bLyr1'])
        output2 = scoreA (self.x2_, self.params_w_['wLyr1'], self.params_b_['bLyr1'])
        output3 = scoreA (self.x3_, self.params_w_['wLyr1'], self.params_b_['bLyr1'])
        output4 = scoreA (self.x4_, self.params_w_['wLyr1'], self.params_b_['bLyr1'])
        output5 = scoreA (self.x5_, self.params_w_['wLyr1'], self.params_b_['bLyr1'])
        output6 = scoreA (self.x6_, self.params_w_['wLyr1'], self.params_b_['bLyr1'])
        output7 = scoreA (self.x7_, self.params_w_['wLyr1'], self.params_b_['bLyr1'])
        output8 = scoreA (self.x8_, self.params_w_['wLyr1'], self.params_b_['bLyr1'])
        output9 = scoreA (self.x9_, self.params_w_['wLyr1'], self.params_b_['bLyr1'])
        output10 = scoreA (self.x10_, self.params_w_['wLyr1'], self.params_b_['bLyr1'])
        output11 = scoreA (self.x11_, self.params_w_['wLyr1'], self.params_b_['bLyr1'])
        output12 = scoreA (self.x12_, self.params_w_['wLyr1'], self.params_b_['bLyr1'])
        output13 = scoreA (self.x13_, self.params_w_['wLyr1'], self.params_b_['bLyr1'])
        output14 = scoreA (self.x14_, self.params_w_['wLyr1'], self.params_b_['bLyr1'])
        output15 = scoreA (self.x15_, self.params_w_['wLyr1'], self.params_b_['bLyr1'])
        output16 = scoreA (self.x16_, self.params_w_['wLyr1'], self.params_b_['bLyr1'])
        print('Output_i')
        print(output1.get_shape().as_list())
        concatenated=output1
        for i in range(2,self.imageRot_+1):
            concatenated=tf.concat([concatenated, eval('output%d'% (i))], 3)
        print('Concatenated')
        print(concatenated.get_shape().as_list())
        
        # 1)  
        convLyr_12_conv = conv2d (concatenated, self.params_w_['wLyr2'], self.params_b_['bLyr2'])
        convLyr_12_relu = tf.nn.relu(convLyr_12_conv)
        print('Conv1')
        print(convLyr_12_relu.get_shape().as_list())
        convLyr_12_pool = maxpool2d(convLyr_12_relu, k=2)
        print('Pool1')
        print(convLyr_12_pool.get_shape().as_list())
        
        # 2)
        convLyr_22_conv = conv2d(convLyr_12_pool, self.params_w_['wLyr3'], self.params_b_['bLyr3'])
        convLyr_22_relu = tf.nn.relu(convLyr_22_conv)
        print('Conv2')
        print(convLyr_22_relu.get_shape().as_list())
        convLyr_22_pool = maxpool2d(convLyr_22_relu, k=2)
        print('Pool2')
        print(convLyr_22_pool.get_shape().as_list())
        
        # 3)
        convLyr_32_conv = conv2d(convLyr_22_pool, self.params_w_['wLyr3'], self.params_b_['bLyr3'])
        convLyr_32_relu = tf.nn.relu(convLyr_32_conv)
        print('Conv3')
        print(convLyr_32_relu.get_shape().as_list())
        convLyr_32_pool = maxpool2d(convLyr_32_relu, k=2)
        print('Pool3')
        print(convLyr_32_pool.get_shape().as_list())
        

        fcLyr_ = tf.nn.dropout(convLyr_32_pool, self.keepProb_)
        print('DropOut')
        print(fcLyr_.get_shape().as_list())

        fcLyr_ = tf.contrib.layers.flatten(fcLyr_)
        print('DropOut')
        print(fcLyr_.get_shape().as_list())
        self.LyrSize_ = fcLyr_.get_shape().as_list()[1]
        [self.params_w_, self.params_b_] = ConvNet.parameters(self)
     
        # 4) Fully Connected
        fcLyr_1 = tf.add(tf.matmul(fcLyr_, self.params_w_['wFCh1']), self.params_b_['bFCh1'])
        fcLyr_1 = tf.nn.relu(fcLyr_1)
        print('Full1')
        print(fcLyr_1.get_shape().as_list())

        fcLyr_2 = tf.add(tf.matmul(fcLyr_1, self.params_w_['wFCh2']), self.params_b_['bFCh2'])
        fcLyr_2 = tf.nn.relu(fcLyr_2)
        print('Full2')
        print(fcLyr_2.get_shape().as_list())

        fcLyr_3 = tf.add(tf.matmul(fcLyr_2, self.params_w_['wFCh3']), self.params_b_['bFCh3'])
        fcLyr_3 = tf.nn.relu(fcLyr_3)
        print('Full3')
        print(fcLyr_3.get_shape().as_list())
        
        fcLyr_4 = tf.add(tf.matmul(fcLyr_3, self.params_w_['wFCh4']), self.params_b_['bFCh4'])
        fcLyr_4 = tf.nn.relu(fcLyr_4)
        print('Full4')
        print(fcLyr_4.get_shape().as_list())
        
        fcLyr_5 = tf.add(tf.matmul(fcLyr_4, self.params_w_['wFCh5']), self.params_b_['bFCh5'])
        fcLyr_5 = tf.nn.relu(fcLyr_5)
        print('Full5')
        print(fcLyr_5.get_shape().as_list())
    
        fcLyr_6 = tf.add(tf.matmul(fcLyr_5, self.params_w_['wFCh6']), self.params_b_['bFCh6'])
        fcLyr_6 = tf.nn.relu(fcLyr_6)
        print('Full6')
        print(fcLyr_6.get_shape().as_list())

        netOut = tf.add(tf.matmul(fcLyr_6, self.params_w_['wOut']), self.params_b_['bOut'])
        print('Out')
        print(netOut.get_shape().as_list())
        return netOut
        
    #------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ 

    def costs(self):
        
        score_split = tf.split( axis=1, num_or_size_splits=self.outLyrSize_, value=self.score_ )
        label_split = tf.split( axis=1, num_or_size_splits=self.outLyrSize_, value=self.y_  )
        weight_split =tf.split( axis=0, num_or_size_splits=self.outLyrSize_, value=self.weightClass_)
        
        total = 0.0
        for i in range (len(score_split)):
            #a=weight_split[i][1]/weight_split[i][0]
            #print(a)
            # your class weights
            #class_weights = [weight_split[i][0], weight_split[i][1]]
            # deduce weights for batch samples based on their true label
            #weights = tf.reduce_sum(class_weights * label_split[i], axis=1)
            # compute your (unweighted) softmax cross entropy loss
            unweighted_losses=tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(logits=score_split[i],
                                                                    targets=label_split[i],
                                                                    pos_weight=weight_split[i][0]))
            #unweighted_losses = tf.losses.sparse_softmax_cross_entropy(
            #    logits=score_split[i], labels=label_split[i],weights=weight_split[i][1])
            # apply the weights, relying on broadcasting of the multiplication
            #weighted_losses = unweighted_losses * weights
            # reduce the result to get your final loss
            loss = unweighted_losses#tf.reduce_mean(weighted_losses)
            total += loss
        return total
        
    #------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------   

    def optimizer(self):
        return tf.train.AdamOptimizer(learning_rate = self.lr_).minimize(self.cost_)
    
    def predicter(self):
        
        return [self.score_, tf.sigmoid(self.score_)]


    #------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    def accuracy(self): 
    
        score_split = tf.split( axis=1, num_or_size_splits=self.outLyrSize_,value=self.score_ )
        label_split = tf.split( axis=1, num_or_size_splits=self.outLyrSize_, value=self.y_     )
   
        correct_pred1  = tf.equal(tf.argmax(score_split[0],1), tf.argmax(label_split[0],1))  
        correct_pred2  = tf.equal(tf.argmax(score_split[1],1), tf.argmax(label_split[1],1))   
        correct_pred3  = tf.equal(tf.argmax(score_split[2],1), tf.argmax(label_split[2],1))  
        correct_pred4  = tf.equal(tf.argmax(score_split[3],1), tf.argmax(label_split[3],1))  
        correct_pred5  = tf.equal(tf.argmax(score_split[4],1), tf.argmax(label_split[4],1))  
        correct_pred6  = tf.equal(tf.argmax(score_split[5],1), tf.argmax(label_split[5],1))  
        correct_pred7  = tf.equal(tf.argmax(score_split[6],1), tf.argmax(label_split[6],1))  
        correct_pred8  = tf.equal(tf.argmax(score_split[7],1), tf.argmax(label_split[7],1))  
        correct_pred9  = tf.equal(tf.argmax(score_split[8],1), tf.argmax(label_split[8],1))  
        correct_pred10  = tf.equal(tf.argmax(score_split[9],1), tf.argmax(label_split[9],1))  
        correct_pred11  = tf.equal(tf.argmax(score_split[10],1), tf.argmax(label_split[10],1))  
        correct_pred12  = tf.equal(tf.argmax(score_split[11],1), tf.argmax(label_split[11],1))  
        correct_pred13  = tf.equal(tf.argmax(score_split[12],1), tf.argmax(label_split[12],1))  
        correct_pred14  = tf.equal(tf.argmax(score_split[13],1), tf.argmax(label_split[13],1))  
        correct_pred15  = tf.equal(tf.argmax(score_split[14],1), tf.argmax(label_split[14],1))  
        correct_pred16  = tf.equal(tf.argmax(score_split[15],1), tf.argmax(label_split[15],1))
        correct_pred17  = tf.equal(tf.argmax(score_split[16],1), tf.argmax(label_split[16],1))
        
        return [correct_pred1 , correct_pred2, correct_pred3, correct_pred4 ,correct_pred5, correct_pred6 , correct_pred7, correct_pred8 , correct_pred9,
                correct_pred10 , correct_pred11, correct_pred12 , correct_pred13, correct_pred14 , correct_pred15, correct_pred16 , correct_pred17]
        
   #------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------    

    def __init__(self,x,y,lr,
                 lyr1FilterNo,lyr2FilterNo,
                 fcHidLyrSize1,fcHidLyrSize2,fcHidLyrSize3,fcHidLyrSize4,fcHidLyrSize5,fcHidLyrSize6,
                 inLyrSize,outLyrSize, keepProb, weight_class, imageRot):
        
        self.x1_            = x[1]
        self.x2_            = x[2]
        self.x3_            = x[3]
        self.x4_            = x[4]
        self.x5_            = x[5]
        self.x6_            = x[6]
        self.x7_            = x[7]
        self.x8_            = x[8]
        self.x9_            = x[9]
        self.x10_            = x[10]
        self.x11_            = x[11]
        self.x12_           = x[12]
        self.x13_            = x[13]
        self.x14_            = x[14]
        self.x15_            = x[15]
        self.x16_            = x[16]

        self.y_            = y
        self.lr_           = lr
        self.outLyrSize_   = outLyrSize
        self.inLyrSize_    = inLyrSize
        self.lyr1FilterNo_ = lyr1FilterNo
        self.lyr2FilterNo_ = lyr2FilterNo
        self.fcHidLyrSize1_ = fcHidLyrSize1
        self.fcHidLyrSize2_ = fcHidLyrSize2
        self.fcHidLyrSize3_ = fcHidLyrSize3
        self.fcHidLyrSize4_ = fcHidLyrSize4
        self.fcHidLyrSize5_ = fcHidLyrSize5
        self.fcHidLyrSize6_ = fcHidLyrSize6
        self.keepProb_      = keepProb
        self.weightClass_   = weight_class
        self.LyrSize_ =1
        self.imageRot_ = imageRot

        [self.params_w_, self.params_b_] = ConvNet.parameters(self) # initialization and packing the parameters
        self.score_                      = ConvNet.scoreB     (self)  # Computing the score function
        self.cost_                       = ConvNet.costs     (self)  # Computing the cost function
        self.optimizer_                  = ConvNet.optimizer (self)  # Computing the update function
        self.predicter_                  = ConvNet.predicter (self)  # Computing the update function
        [self.perf_1, self.perf_2, self.perf_3, self.perf_4,self.perf_5,
         self.perf_6, self.perf_7, self.perf_8,self.perf_9, self.perf_10,
         self.perf_11, self.perf_12,self.perf_13, self.perf_14,self.perf_15,
         self.perf_16, self.perf_17] = ConvNet.accuracy  (self)  # performance


In [ ]:
if __name__ == '__main__':
    
    lyr1FilterNo = 16 
    lyr2FilterNo = 32 
    
    fcHidLyrSize1 = 15000
    fcHidLyrSize2 = 7000
    fcHidLyrSize3 = 2000
    fcHidLyrSize4 = 600
    fcHidLyrSize5 = 150
    fcHidLyrSize6 = 37
    inLyrSize    = 660 * 512
    outLyrSize   = 17
    lr           = 0.001
    batch_size   = 1
   
    dropout      = 0.5
    keepProb     = tf.placeholder(tf.float32)
    x            = tf.placeholder(tf.float32)
    y            = tf.placeholder(tf.float32, [None, outLyrSize])
    weight_class = tf.placeholder(tf.float32)
    imageRot     = 15
    
    ConvNet_class = ConvNet(x,y,
                            lr,
                            lyr1FilterNo,lyr2FilterNo,
                            fcHidLyrSize1,fcHidLyrSize2,fcHidLyrSize3,fcHidLyrSize4,fcHidLyrSize5,fcHidLyrSize6,
                            inLyrSize,outLyrSize,
                            keepProb,
                            weight_class, imageRot)
    initVar = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(initVar)  
        index = 0 

        for batch_i in range(4000):
            trData_i=dict()
            trLabel_i = []
            print(index)
            for i in range(16):
                trXi = trX[i][index : index + batch_size]
                trData_i[i]=trXi
            trLabel_i.append( trY[ index : index + batch_size ] )
            
            index += batch_size
            
            if index > ( len(trX[0]) -1 ):
                index = 0
                

            trLabel_i = np.reshape( trLabel_i, ( -1, 17 ) )
            #print(trLabel_i)
            sess.run( ConvNet_class.optimizer_ ,
                     feed_dict = {x:[values for values in trData_i.values()],
                                  y:trLabel_i,
                                  keepProb:dropout,
                                  weight_class : np.asarray([x for d in class_X.values() for x in d.values()])})
            #print(trLabel_i)
            if index== 0: 
            
                cost_tr = sess.run(ConvNet_class.cost_,
                                   feed_dict={x:[values for values in trData_i.values()],
                                              y:trLabel_i,   keepProb: 1.,
                                             weight_class : np.asarray([x for d in class_X.values() for x in d.values()])})
                cost_te = sess.run(ConvNet_class.cost_,
                                   feed_dict={x:[values for values in teX.values()],
                                              y:teY,  keepProb: 1.,
                                             weight_class : np.asarray([x for d in class_X.values() for x in d.values()])})
                
                # test accuracy
                [accu1, accu2,accu3, accu4,accu5, accu6,accu7, accu8,accu9,
                accu10,accu11, accu12,accu13, accu14,accu15, accu16,accu17]  = sess.run([ConvNet_class.perf_1, ConvNet_class.perf_2,
                                                                                         ConvNet_class.perf_3, ConvNet_class.perf_4,
                                                                                         ConvNet_class.perf_5, ConvNet_class.perf_6,
                                                                                         ConvNet_class.perf_7, ConvNet_class.perf_8,
                                                                                         ConvNet_class.perf_9, ConvNet_class.perf_10,
                                                                                         ConvNet_class.perf_11,ConvNet_class.perf_12,
                                                                                         ConvNet_class.perf_13,ConvNet_class.perf_14,
                                                                                         ConvNet_class.perf_15,ConvNet_class.perf_16, 
                                                                                         ConvNet_class.perf_17] ,
                                                                                        feed_dict={x:[values for values in teX.values()],
                                                                                                   y:teY,  keepProb: 1.})
                numOfposit = 0.0
                for tt in range(accu1.shape[0]):
                    if accu1[tt] == accu2[tt] and accu1[tt] == True:
                        numOfposit += 1
                test_accu = numOfposit / accu1.shape[0]

                print("%4d, cost_tr: %4.2g , cost_te: %4.2g , testAccu: %4.2g "% ( batch_i , cost_tr , cost_te ,test_accu ) )
                if cost_tr==0:
                    trData_i=dict()
                    trLabel_i = []
                    for i in range(16):
                        trXi = trX[i][0 : 2]
                        trData_i[i]=trXi
                    trLabel_i.append( trY[0 : 2 ] )
                    trLabel_i = np.reshape( trLabel_i, ( -1, 17 ) )
                    
                    [Score_te,Pred_te] = sess.run(ConvNet_class.predicter_,
                       feed_dict={x:[values for values in trData_i.values()],
                                  y:trLabel_i,  keepProb: 1.,
                                  weight_class : np.asarray([x for d in class_X.values() for x in d.values()])})
                    print(trY[:2])
                    print(Pred_te[:2].astype(int))
                    print(Score_te[:2])
                
                [Score_te,Pred_te] = sess.run(ConvNet_class.predicter_,
                                              feed_dict={x:[values for values in teX.values()],
                                                         y:teY,  keepProb: 1.,
                                                         weight_class : np.asarray([x for d in class_X.values() for x in d.values()])})
                print(teY)
                print(Pred_te.astype(int))
                print(Score_te)
    
    Pred_te = sess.run(ConvNet_class.predicter_,
                       feed_dict={x:[values for values in teX.values()],
                                  y:teY,  keepProb: 1.,
                                  weight_class : np.asarray([x for d in class_X.values() for x in d.values()])})
    print(teY)
    print(Pred_te)
                

## 